<a href="https://colab.research.google.com/github/hemanth346/eva4/blob/master/S4/EVA4_S4_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
def conv_block(in_channels, out_channels, dropout=0, *args, **kwargs):
    if dropout:
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, *args, **kwargs),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout2d(dropout)
            )
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, *args, **kwargs),
        nn.BatchNorm2d(out_channels),
        nn.ReLU()
        )


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = conv_block(in_channels=1, out_channels=16, kernel_size=3, padding=1, bias=False, dropout=0.01)             # i:28x28x1     k:8	    O:28x28x16      RF:3
        self.conv2 = conv_block(in_channels=16, out_channels=16, kernel_size=3, bias=False, dropout=0.01)                       # i:26x26x16    k:8	    O:26x26x16      RF:5
        self.pool = nn.MaxPool2d(2, 2)                                                                                          # i:26x26x16    k:NA    O:13x13x16      RF:6

        self.conv3 = conv_block(in_channels=16, out_channels=16, kernel_size=3, padding=1, bias=False, dropout=0.01)            # i:13x13x16    k:16    O:13x13x16      RF:10
        self.conv4 = conv_block(in_channels=16, out_channels=32, kernel_size=3, bias=False, dropout=0.01)                       # i:13x13x16    k:32    O:11x11x32      RF:14
        # self.pool = nn.MaxPool2d(2, 2)                                                                                        # i:11x11x32    k:NA    O:5x5x32        RF:16 ## 2nd pool

        self.conv5 = conv_block(in_channels=32, out_channels=16, kernel_size=3, padding=1, bias=False, dropout=0.01)            # i:5x5x32      k:16    O:5x5x16        RF:24
        self.conv6 = conv_block(in_channels=16, out_channels=32, kernel_size=3, bias=False, dropout=0.01)                       # i:5x5x16      k:32    O:3x3x32        RF:32
        # self.pool = nn.MaxPool2d(2, 2)                                                                                        # i:3x3x32      k:NA    O:3x3x32        RF:35 ## 3rd pool
        
        self.gap = nn.AdaptiveAvgPool3d((10,1,1))                                                                               # i:3x3x32      k:NA    O:1x1x10       RF:35


    def forward(self, x):
        x = self.pool(self.conv2(self.conv1(x)))
        x = self.pool(self.conv4(self.conv3(x)))
        x = self.pool(self.conv6(self.conv5(x)))
        x = self.gap(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
try:
    from torchsummary import summary
except:
    !pip install torchsummary
    from torchsummary import summary

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             144
       BatchNorm2d-2           [-1, 16, 28, 28]              32
              ReLU-3           [-1, 16, 28, 28]               0
         Dropout2d-4           [-1, 16, 28, 28]               0
            Conv2d-5           [-1, 16, 26, 26]           2,304
       BatchNorm2d-6           [-1, 16, 26, 26]              32
              ReLU-7           [-1, 16, 26, 26]               0
         Dropout2d-8           [-1, 16, 26, 26]               0
         MaxPool2d-9           [-1, 16, 13, 13]               0
           Conv2d-10           [-1, 16, 13, 13]           2,304
      BatchNorm2d-11           [-1, 16, 13, 13]              32
             ReLU-12           [-1, 16, 13, 13]               0
        Dropout2d-13           [-1, 16, 13, 13]               0
           Conv2d-14           [-1, 32,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [0]:
from tqdm import tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    pbar = tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    print('\nTraining Set: Accuracy: {}/{} ({:.2f}%)\n'.format(
        correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest Set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
    
for epoch in range(1, 21):
    print('Epoch :', epoch)
    if epoch < 10:
        optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    elif 10 < epoch < 15:
        optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    else:
        optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch : 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.




Training Set: Accuracy: 57294/60000 (95.49%)


Test Set: Average loss: 0.0582, Accuracy: 9868/10000 (98.68%)

Epoch : 2




Training Set: Accuracy: 59046/60000 (98.41%)


Test Set: Average loss: 0.0396, Accuracy: 9916/10000 (99.16%)

Epoch : 3




Training Set: Accuracy: 59228/60000 (98.71%)


Test Set: Average loss: 0.0360, Accuracy: 9907/10000 (99.07%)

Epoch : 4


...



Training Set: Accuracy: 59814/60000 (99.69%)


Test Set: Average loss: 0.0205, Accuracy: 9943/10000 (99.43%)

Epoch : 17




Training Set: Accuracy: 59848/60000 (99.75%)


Test Set: Average loss: 0.0203, Accuracy: 9946/10000 (99.46%)

Epoch : 18




Training Set: Accuracy: 59825/60000 (99.71%)


Test Set: Average loss: 0.0201, Accuracy: 9945/10000 (99.45%)

Epoch : 19




Training Set: Accuracy: 59838/60000 (99.73%)


Test Set: Average loss: 0.0204, Accuracy: 9946/10000 (99.46%)

Epoch : 20




Training Set: Accuracy: 59856/60000 (99.76%)


Test Set: Average loss: 0.0203, Accuracy: 9945/10000 (99.45%)

